# 8.4 Parsing with Context-Free Grammar

In [2]:
import nltk
from nltk import CFG

### Recursive Descent Parsing

In [18]:
grammar1 = nltk.data.load('file:mygrammar.cfg')
rd_parser = nltk.RecursiveDescentParser(grammar1, trace=2)
sent = 'Mary saw a dog'.split()
for t in rd_parser.parse(sent):
    print t

Parsing u'Mary saw a dog'
    [ * S ]
  E [ * NP VP ]
  E [ * 'John' VP ]
  E [ * 'Mary' VP ]
  M [ 'Mary' * VP ]
  E [ 'Mary' * V NP ]
  E [ 'Mary' * 'saw' NP ]
  M [ 'Mary' 'saw' * NP ]
  E [ 'Mary' 'saw' * 'John' ]
  E [ 'Mary' 'saw' * 'Mary' ]
  E [ 'Mary' 'saw' * 'Bob' ]
  E [ 'Mary' 'saw' * Det N ]
  E [ 'Mary' 'saw' * 'a' N ]
  M [ 'Mary' 'saw' 'a' * N ]
  E [ 'Mary' 'saw' 'a' * 'man' ]
  E [ 'Mary' 'saw' 'a' * 'dog' ]
  M [ 'Mary' 'saw' 'a' 'dog' ]
  + [ 'Mary' 'saw' 'a' 'dog' ]
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))
  E [ 'Mary' 'saw' 'a' * 'cat' ]
  E [ 'Mary' 'saw' 'a' * 'telescope' ]
  E [ 'Mary' 'saw' 'a' * 'park' ]
  E [ 'Mary' 'saw' * 'an' N ]
  E [ 'Mary' 'saw' * 'the' N ]
  E [ 'Mary' 'saw' * 'my' N ]
  E [ 'Mary' 'saw' * Det N PP ]
  E [ 'Mary' 'saw' * 'a' N PP ]
  M [ 'Mary' 'saw' 'a' * N PP ]
  E [ 'Mary' 'saw' 'a' * 'man' PP ]
  E [ 'Mary' 'saw' 'a' * 'dog' PP ]
  M [ 'Mary' 'saw' 'a' 'dog' * PP ]
  E [ 'Mary' 'saw' 'a' 'dog' * P NP ]
  E [ 'Mary' 'saw' 'a

In [19]:
sr_parser = nltk.ShiftReduceParser(grammar1, trace=2)
sent = 'Mary saw a dog'.split()
for tree in sr_parser.parse(sent):
    print(tree)

Parsing u'Mary saw a dog'
    [ * Mary saw a dog]
  S [ 'Mary' * saw a dog]
  R [ NP * saw a dog]
  S [ NP 'saw' * a dog]
  R [ NP V * a dog]
  S [ NP V 'a' * dog]
  R [ NP V Det * dog]
  S [ NP V Det 'dog' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


## Well-Formed Substring Tables

In [37]:
groucho_grammar = CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

In [38]:
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

In [39]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

In [40]:
def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    if trace:
                        print "[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                        (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end)
    return wfst

In [41]:
def display(wfst, tokens):
    print '\nWFST ' + ' '.join([("%-4d" % i) for i in range(1, len(wfst))])
    for i in range(len(wfst)-1):
        print "%d  " % i,
        for j in range(1, len(wfst)):
            print "%-4s" % (wfst[i][j] or '.'),
        print

In [42]:
tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)


WFST 1    2    3    4    5    6    7   
0   NP   .    .    .    .    .    .   
1   .    V    .    .    .    .    .   
2   .    .    Det  .    .    .    .   
3   .    .    .    N    .    .    .   
4   .    .    .    .    P    .    .   
5   .    .    .    .    .    Det  .   
6   .    .    .    .    .    .    N   


In [34]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)


WFST 1    2    3    4    5    6    7   
0   NP   .    .    S    .    .    S   
1   .    V    .    VP   .    .    VP  
2   .    .    Det  NP   .    .    .   
3   .    .    .    N    .    .    .   
4   .    .    .    .    P    .    PP  
5   .    .    .    .    .    Det  NP  
6   .    .    .    .    .    .    N   


In [35]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]
